In [14]:
from keras.datasets import mnist
from sklearn.utils import shuffle
import math
import random
import numpy as np
from numpy.core.multiarray import dot


import torch

if torch.cuda.is_available():  
  dev = "cuda:0" 
  print("gpu up")
else:  
  dev = "cpu"  
device = torch.device(dev)




gpu up


In [16]:
# Shuffling the dataset

(train_X, train_y), (test_X, test_y) = mnist.load_data()


train_X, train_y = shuffle(train_X[:1500], train_y[:1500])
test_X, test_y = shuffle(test_X[:500], test_y[:500])


# train_X, train_y = train_X[:500], train_y[:500]
# test_X, test_y = test_X[:100], test_y[:100]

arr = slicingArray(train_X, 4, 7)
trainFeatureVec = normalize(arr)
trainFeatureVec = arr
trainFeatureVec = trainFeatureVec.reshape([*arr.shape[:-2], -1])
trainFeatureVec = np.expand_dims(trainFeatureVec, axis=-1)


# print(trainFeatureVec.shape)
# trainFeatureVec = trainFeatureVec.tolist()


arr = slicingArray(test_X, 4, 7)
testFeatureVec = normalize(arr)
testFeatureVec = arr
testFeatureVec = testFeatureVec.reshape(*arr.shape[:-2], -1)
testFeatureVec = np.expand_dims(testFeatureVec, axis=-1)


train_y = (np.eye(10)[np.array(train_y)])
test_y = (np.eye(10)[np.array(test_y)])
train_y = np.expand_dims(train_y,axis=-1)
test_y = np.expand_dims(test_y, axis=-1)


In [3]:
def centroid(arr):
    moments = {'M00': arr.sum(axis=1).sum(),
               'M01': (np.arange(start=1, stop=arr.shape[1] + 1, step=1) * arr.sum(axis=0).reshape(1, -1)).sum(),
               'M10': (np.arange(start=1, stop=arr.shape[0] + 1, step=1) * arr.sum(axis=1).reshape(1, -1)).sum(),
               'M11': 0}

    x = moments['M10'] / moments['M00'] if moments['M00'] != 0 else 0
    y = moments['M01'] / moments['M00'] if moments['M00'] != 0 else 0
    return [x, y]

threshold = 150  # for Binarize


def Binarize(X):
    X[X < threshold] = 0
    X[X >= threshold] = 1
    return X


# train_X = Binarize(train_X)
# test_X = Binarize(test_X)


def normalize(X):
    mx = np.max(X)
    return X/mx


def imageWin(trainx, r, c):
    temp = []
    for i in range(0, int(trainx.shape[0]), r):
        for j in range(0, int(trainx.shape[1]), c):
            temp.append(centroid(trainx[i:i+r, j:j+c]))
    return np.array(temp)


def slicingArray(trainX, r, c):
    featureVector = []
    for trainx in trainX:
        featureVector.append(imageWin(trainx, r, c))
    return np.array(featureVector)


In [5]:
# Some Fundamental Functions
def listXlist(l1: list, l2: list):
    res = [[0 for i in range(len(l2[0]))] for j in range(len(l1))]
    # print(len(l1),len(l1[0])," ",len(l2),len(l2[0]))
    for i in range(len(l1)):
        for j in range(len(l2[0])):
            for k in range(len(l2)):
                res[i][j] += l1[i][k] * l2[k][j]
    return res


def listMINUSlist(l1: list, l2: list):
    res = [[0 for i in range(len(l2[0]))] for j in range(len(l2))]
    for i in range(len(l2)):
        for j in range(len(l1[0])):
            res[i][j] = l1[i][j] - l2[i][j]
    return res


def listSquare(l: list):
    res = []
    for i in range(len(l)):
        res.append([l[i]**2])
    return res


def mean(k: list):
    return sum([i[0] for i in k]) / len(k)


def get_label(vector: np.array):
    labels = []
    for i in range(len(vector)):
        mx = 0
        idx = 0
        for j in range(len(vector[0])):
            if(vector[i][j] > mx):
                mx = vector[i][j]
                idx = j
        labels.append(idx)
    return np.array(labels)


def normalizeOut(vector: np.array):
    mx = vector[get_label(vector)]
    vector = [vector[i]/mx for i in range(len(vector))]
    return vector


def accuracy(target: np.array, predictions: np.array):
    trueValues = get_label(target)
    preds = get_label(predictions)
    cnt = 0
    for i in range(len(trueValues)):
        if trueValues[i] == preds[i]:
            cnt += 1

    return cnt / float(len(trueValues)) * 100.0

def sigmoid(x): return 1/(1+np.exp(-x))

In [6]:
# Fully connected NN
class NN:
    def __init__(self, m: int, n: np.array, activation: str, e: int, lr: float, outShape: int):
        self.neurons = n                                                    # Neurons List to specify number of neurons in each layer
        self.neuronsListBefore = []                                         # Net values
        self.neuronsListAfter = []                                          # Neurons output values
        self.layers = m                                                     # Number of layers
        self.weights = dict([(x, [0]) for x in range(self.layers + 1)])     # Dictionary to save weights of each layer
        self.bias = dict([(x, [0]) for x in range(self.layers + 1)])        # Dictionary to save bias of each layer
        self.gradsWeights = []                                              # Dictionary to save weights gradients of each layer
        self.gradsBias = []                                                 # Dictionary to save bias gradients of each layer
        self.activationFn = activation                                      # String to specify layers' activation function
        self.learning_rate = lr                                         
        self.epochs = e
        self.outputShape = outShape                                         # Number of neurons in output layer

    # MSE
    def __costFunc(self, y, preds):
        return 0.5 * listSquare(listMINUSlist(y, preds)) /len(y)

    def __activation(self, x: float, act: str):
       
        if act == "sigmoid":
            return sigmoid(x)
        if act == "tanh":
            return np.tanh(x)
        if act == "linear":
            return x
        if act=="relu":
            return np.maximum(x*0, x),
        if act=="leaky_relu":
            return np.maximum(0.1 * x, x)
        
    def __derivatives(self, out: float, act: str):
        if act == "sigmoid":
            return sigmoid(out) * (1-sigmoid(out))
        if act=="tanh":
            return 1 - (out ** 2)
        if act=="linear":
            return np.ones(out.shape)
        if act=="relu":
            re = np.ones(out.shape)
            re[out<0] = out[out<0] * 0
            return re
        if act=="leaky_relu":
            re = np.ones(out.shape)
            re[out<0] = out[out<0] * 0.1
            return re

    def __calcMSE(self,pred,y):
      pred= pred.squeeze(-1)
      y = y.squeeze(-1)
      return np.mean(((y-pred)* (y-pred)).sum(axis=-1))

    def __initializeWeightsBias(self, input: np.array):
        self.weights[0] = np.array([[random.random() for i in range(len(input))]
                           for j in range(self.neurons[0])],dtype=np.double)
        self.bias[0] = np.array([[random.random()]
                        for i in range(self.neurons[0])],dtype=np.double)

        for i in range(1, self.layers):
            self.weights[i] = np.array([[random.random() for i in range(
                self.neurons[i - 1])] for j in range(self.neurons[i])],dtype=np.double)
            self.bias[i] = np.array([[random.random()]
                            for i in range(self.neurons[i])],dtype=np.double)

        self.weights[self.layers] = np.array([[random.random() for i in range(
            self.neurons[self.layers - 1])] for j in range(self.outputShape)],dtype=np.double)
        self.bias[self.layers] = np.array([[random.random()]
                                  for i in range(self.outputShape)],dtype=np.double)

    def __zero_grads(self, input: list):
        self.gradsWeights = dict([(x, [0]) for x in range(self.layers + 1)])
        self.gradsBias = dict([(x, [0]) for x in range(self.layers + 1)])
        self.gradsWeights[0] = np.array([[0 for i in range(len(input))]
                                for j in range(self.neurons[0])],dtype=np.double)
        self.gradsBias[0] = np.array([[0] for i in range(self.neurons[0])],dtype=np.double)

        for i in range(1, self.layers):
            self.gradsWeights[i] = np.array([
                [0 for _ in range(self.neurons[i - 1])] for j in range(self.neurons[i])],dtype=np.double)
            self.gradsBias[i] = np.array([[0] for i in range(self.neurons[i])],dtype=np.double)

        self.gradsWeights[self.layers] = np.array([[0 for i in range(
            self.neurons[self.layers - 1])] for j in range(self.outputShape)],dtype=np.double)
        self.gradsBias[self.layers] = np.array([[0] for i in range(self.outputShape)],dtype=np.double)

    def __feedForward(self, input: np.array, layerIdx: int) -> np.array:
        out = []
        x = []
        x = (self.weights[layerIdx]@ input) + self.bias[layerIdx]
        self.neuronsListBefore.append(x)
        out = self.__activation(x, self.activationFn)
        self.neuronsListAfter.append(out)
        return out

    def __propagate(self, x: np.array):
        self.neuronsListAfter.append(x)
        out = self.__feedForward(x, 0)
        for i in range(1, self.layers + 1):
            out = self.__feedForward(out, i)
        return out

    def __backPropagation(self, target: np.array, outputLayer: np.array):
        self.deltas = dict([(x, [0]) for x in range(self.layers + 1)])
        self.deltas[self.layers] = ( -1* (target-outputLayer) )* (self.__derivatives(
                self.neuronsListBefore[self.layers], self.activationFn))
        self.gradsBias[self.layers]= self.deltas[self.layers]
        self.gradsWeights[self.layers] = self.deltas[self.layers] @ self.neuronsListAfter[self.layers].T
       

        for layer in reversed(range(0, self.layers)):

            # (neuron, 1) (neuron, neuronsBefore ) -> (neurons,Before)
            self.deltas[layer] = (self.weights[layer + 1].T @ self.deltas[layer+1] )* self.__derivatives(
                    self.neuronsListBefore[layer], self.activationFn)
            
            self.gradsBias[layer] = self.deltas[layer]
            self.gradsWeights[layer] = self.deltas[layer] @ self.neuronsListAfter[layer].T
            

    def __updateWeightsBias(self):
        for layer in range(self.layers + 1):
            self.bias[layer] -= self.learning_rate * self.gradsBias[layer]
            self.weights[layer] -= self.learning_rate * self.gradsWeights[layer] 

    def fit(self, trainX: np.array, trainY: np.array):
        self.__initializeWeightsBias(trainX[0])
        self.__zero_grads(trainX[0])
        errs = []
        for epoch in range(self.epochs):
            print("epoch: ", epoch)
            outs= []
            for i in range(len(trainX)):
                self.neuronsListBefore = []
                self.neuronsListAfter = []
                outputLayer = self.__propagate(trainX[i])
                outs.append(outputLayer.tolist())
                self.__backPropagation(trainY[i], outputLayer)
                self.__updateWeightsBias()
                self.__zero_grads(trainX[i])
            errs.append(self.__calcMSE(np.array(outs),trainY))
        return errs

    def predict(self, testX: np.array):
        preds = []
        for i in range(len(testX)):
            preds.append(self.__propagate(testX[i]))
        return preds


In [20]:
nn = NN(2, [56, 16], 'sigmoid', 100, 0.003, 10)

errs = nn.fit(np.array(trainFeatureVec,dtype=np.double), np.array(train_y))

TypeError: float() argument must be a string or a number, not 'StandardScaler'

In [12]:
print(errs)

[8.992172945843958, 8.991643227722813, 8.9910019919409, 8.990201715623773, 8.989160487906291, 8.987722562205112, 8.985547227443922, 8.981705336251082, 8.972340664908018, 8.876877442994381, 8.242251377059317, 8.207867147380155, 8.206536968547862, 8.20543568622867, 8.204029490264542, 8.202023580214176, 8.198803798177966, 8.192369161287848, 8.168531037917973, 7.687141690027643, 7.407943300007213, 7.403179114206905, 7.397893729918127, 7.384269323413612, 7.1373329593234525, 6.590856126554943, 6.571529613009467, 6.401403210086733, 5.783244556667121, 5.76808518076942, 5.7671040573812835, 5.766678315342159, 5.76630434039467, 5.765877595921404, 5.765341346134113, 5.7646272463401536, 5.763617734208768, 5.762071048249105, 5.759385613369498, 5.753510590424511, 5.728948436082863, 5.2175769023473455, 4.954412574173858, 4.952256421693551, 4.951960609547009, 4.951874468545752, 4.951831116107718, 4.951797999062893, 4.9517669515737825, 4.95173561035282, 4.951703190499914, 4.951669365958438, 4.9516339451

In [11]:
testpreds = nn.predict(np.array(testFeatureVec,dtype=np.double))
# print(get_label(np.array(test_y[:6])))
print("Accuracy: ", accuracy(test_y, testpreds), "%")

Accuracy:  8.6 %


In [10]:
print(get_label(testpreds))

[6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 